In [ ]:
pip install pandas matplotlib seaborn scikit-learn xgboost

In [ ]:
import pandas as pd

PROCESSED_FILES_FOLDER = '../data/processed'
OUTPUT_PATH = f'{PROCESSED_FILES_FOLDER}/full_matches_train_data.csv'


## Definindo as colunas que serão usadas para treinar o modelo
team_map_results_columns_to_add  = ["PlusMinus", "Adr", "Kast", "Rating"]
team_map_results_columns_to_drop  = []

map_results = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in team_map_results_columns_to_add
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

kast_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Kast"]
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

adr_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in range(1, 6) 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Adr"]
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

### Listando as colunas que serão usadas das partidas, como: Jogadores, Mortes, Assistências etc.
matches_train_data_columns = ['eventId', 'matchId', 'mapBestOf'] + map_results

print(f"Reading data from {PROCESSED_FILES_FOLDER}/matches_results.csv and {PROCESSED_FILES_FOLDER}/matches.csv")
matches_results_train_data = pd.read_csv('../data/raw/matches_results.csv')
matches_train_data = pd.read_csv('../data/raw/matches.csv')

matches_train_data = matches_train_data[matches_train_data_columns]

matches_results_train_data_columns = ['eventId', 'matchId', 'TeamOneScore', 'TeamTwoScore', 'teamOneWon', 'teamTwoWon']
matches_results_train_data = matches_results_train_data[[col for col in matches_results_train_data.columns if any(s in col for s in matches_results_train_data_columns)]]

## Combinando os dados gerais das partidas com os detalhes das partidas
full_matches_train_data = pd.merge(matches_results_train_data, matches_train_data, on='matchId', how='inner')

## Data Wrangling
full_matches_train_data.fillna(0, inplace=True)
full_matches_train_data.replace("Not Available", 0, inplace=True)
full_matches_train_data.drop(['eventId_y', 'eventId_x', 'matchId'], axis=1, inplace=True)

full_matches_train_data.drop(team_map_results_columns_to_drop, axis=1, inplace=True)
full_matches_train_data.drop(['mapBestOf'], axis=1, inplace=True)

for column in kast_columns:
    full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x.strip('%')) / 100, 2) if str(x) != '0' else 0)

for column in adr_columns:
    full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x) / 100, 2))
    
full_matches_train_data.to_csv(OUTPUT_PATH, index=False)
print(f"Data saved to {OUTPUT_PATH}")